## Домашняя работа 10

ФИО: Филимонов Степан

Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip

1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)
2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).
Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [1]:
import os
import sys
import time
import torch
import random
import string
import warnings
import datetime
import progressbar

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch import nn
from tqdm import tqdm
from torch import device, cuda
from transformers import BertTokenizer
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

warnings.filterwarnings('ignore')

/Users/stepanfilimonov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
BATCH_SIZE = 32
DEVICE = device("cuda:0" if cuda.is_available() else "cpu")
print(DEVICE)

cpu


In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', do_lower_case=True)

In [4]:
df = pd.read_csv('all_accents.tsv', delimiter='\t', names=['title', 'accent'])
df.sample(5)

,title,accent
742837,натовское,н^атовское
494017,иннервирую,иннерв^ирую
438439,застрачивает,застр^ачивает
1416429,солжет,солж^ет
225796,выпестовалось,в^ыпестовалось


У нас будет задача классификации, где буква на которую падает ударение, есть истенный класс. Для этого в блоке `accent`
оставим только номер гласной, на которую падает уравнение, начиная с *1*. Если значение равно 0 - в слове нет ударения

In [5]:
df['accent'] = [ ( word.find('^') + 1 ) for word in  df['accent']]
NUM_CLASSES = (max(df["accent"]) - min(df["accent"]))
print(f'Колличество данных: {df.shape[0]:,}')
print(f'Кол-во классов: {NUM_CLASSES}')

Колличество данных: 1,680,535
Кол-во классов: 55


In [6]:
MAX_LENGTH = max([len(tokenizer.encode(sent, add_special_tokens=True)) for sent in tqdm(df['title'])])
print('Максимальный размер предложения: ', MAX_LENGTH)

100%|██████████████████████████████| 1680535/1680535 [01:42<00:00, 16366.61it/s]

Максимальный размер предложения:  26


In [7]:
input_ids = []
attention_masks = []

for sent in tqdm(df['title']):
    encoded_dict = tokenizer.encode_plus(sent, add_special_tokens=True, max_length=MAX_LENGTH,
                                         pad_to_max_length=True, return_attention_mask=True,
                                         return_tensors='pt', truncation=True)

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df['accent'])

100%|██████████████████████████████| 1680535/1680535 [02:25<00:00, 11527.35it/s]


In [8]:
dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataset, val_dataset = train_test_split(dataset, train_size=0.5, random_state=42)
print(f'Обучающая выборка: {len(train_dataset):,}')
print(f'Валидация: {len(val_dataset):,}')

Обучающая выборка: 840,267
Валидация: 840,268


In [9]:
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
validation_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)

In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=NUM_CLASSES,
                                                      output_attentions=False, output_hidden_states=False,).to(DEVICE)
optimizer = AdamW(model.parameters(), lr=0.01)
model

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [11]:
EPOCHS = 4
total_steps = len(train_dataloader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [12]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

#### Обучение

In [13]:
for epoch_i in range(0, EPOCHS):
    print(f'======== Epoch {(epoch_i + 1)} / {EPOCHS} ========\nTraining...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 120 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f'\t\tBatch: {step:,}/{len(train_dataloader):,}.\tElapsed: {elapsed} s.')
        
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_labels = batch[2].to(DEVICE)
        model.zero_grad()
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        total_train_loss += output.loss.item()
        output.loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print(f'\tTraining loss: {avg_train_loss:.2f}. Time: {training_time} s.\nRunning Validation...')

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in tqdm(validation_dataloader):
        b_input_ids = batch[0].to(DEVICE)
        b_input_mask = batch[1].to(DEVICE)
        b_labels = batch[2].to(DEVICE)

        with torch.no_grad():
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        total_eval_loss += output.loss.item()
        logits = output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    print(f'\tAccuracy val: {avg_val_accuracy:.2f}. Val Loss: {avg_val_loss:.2f}. Time: {validation_time} s.')

======== Epoch 1 / 4 ========
Training...
		Batch: 120/26,259.	Elapsed: 0:01:25 s.
		Batch: 240/26,259.	Elapsed: 0:02:51 s.
		Batch: 360/26,259.	Elapsed: 0:04:16 s.
		Batch: 480/26,259.	Elapsed: 0:05:56 s.
		Batch: 600/26,259.	Elapsed: 0:07:25 s.
		Batch: 720/26,259.	Elapsed: 0:08:54 s.
		Batch: 840/26,259.	Elapsed: 0:10:27 s.
		Batch: 960/26,259.	Elapsed: 0:11:57 s.
		Batch: 1,080/26,259.	Elapsed: 0:13:59 s.
		Batch: 1,200/26,259.	Elapsed: 0:15:26 s.
		Batch: 1,320/26,259.	Elapsed: 0:16:48 s.
		Batch: 1,440/26,259.	Elapsed: 0:18:11 s.
		Batch: 1,560/26,259.	Elapsed: 0:19:31 s.
		Batch: 1,680/26,259.	Elapsed: 0:20:54 s.
		Batch: 1,800/26,259.	Elapsed: 0:22:18 s.
		Batch: 1,920/26,259.	Elapsed: 0:23:42 s.
		Batch: 2,040/26,259.	Elapsed: 0:25:09 s.
		Batch: 2,160/26,259.	Elapsed: 0:26:43 s.
		Batch: 2,280/26,259.	Elapsed: 0:28:09 s.
		Batch: 2,400/26,259.	Elapsed: 0:29:34 s.
		Batch: 2,520/26,259.	Elapsed: 0:31:00 s.
		Batch: 2,640/26,259.	Elapsed: 0:32:27 s.
		Batch: 2,760/26,259.	Elaps

KeyboardInterrupt: 

**Вывод:** 

Вычисления все полностью выходили бы примерно ~21 час. Исходя из опыта предыдущих домашних работ, если использовать весь массив,
для обучения и валидации модели, то для получения минимального loss достаточно пройти обучение, для экономии ресурса машины я решил до конца недообучать.

P.s. до конца семестра хочу получить доступ к машине с GPU, произведу пересчет данной задачи и дополню решение на портале